# Import Libraries

In [1]:
from gurobipy import *
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import pysd
import numpy as np
import geopandas as gp
import zipfile
import requests
import networkx as nx
import matplotlib.cm
import math
from math import radians, sin, cos, acos
from geopy.distance import geodesic
import requests
from bs4 import BeautifulSoup
from osgeo import ogr, osr
import matplotlib.cm as cmx
import matplotlib.colors as colors
from Circles.circles import circle
from shapely.geometry import Polygon, Point
from descartes import PolygonPatch
from shapely.ops import cascaded_union
import pickle
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import Normalize
import warnings
from copy import deepcopy
import copy
warnings.filterwarnings("ignore")
import time

# Define Functions

In [2]:
def rmw(intensity):
    '''Calculates the radius of maximum wind (in KM) given typhoon intensity (mph)'''
    # below cat 1
    if intensity < 74:
        return 0
    # cat 1 and cat 2
    elif intensity < 111:
        return 55.5
    # cat 3 and cat 4
    elif intensity < 157:
        return 47
    else:
        return 27.8

def r33(intensity):
    '''Calculates the radius of 33 m/s winds (in Km), which defines the area that sustains damage'''
    # below cat 1
    if intensity < 74:
        return 0
    # cat 1 and cat 2
    elif intensity < 111:
        return 74
    elif intensity < 130:
        return 96
    else:
        return 94

def WindSpeedCoeff(radius, intensity):
    '''Returns the wind speed coefficient given the radius (how far is the point from the center of the typhoon) and the intensity of the typhoon'''
    if rmw(intensity) == None:
        return 0
    else:
        return 0.004 * intensity * math.sqrt(((rmw(intensity)/radius)**(3))* math.e**(1-(rmw(intensity)/radius)**(3)))

def distance(a,b):  
    '''shortest distance in km of between two points on earths surface'''
    return geodesic((a[0],a[1]),(b[0],b[1])).km

def DPdemandcoeff(typhooncoords, typhoonintensity, DPcoord):
    '''Inputs the DP coordinate, and the path of the typhoon (location and intensity). It returns the maximum wind speed coefficient'''
    towndistance = []
    towndemand = []
    for i in range(len(typhooncoords)):
        towndistance.append(distance(typhooncoords[i], DPcoord))
    if min(towndistance)*1.60934 <= r33(typhoonintensity[np.argmin(towndistance)]):
        for i in range(len(towndistance)):
            towndemand.append(WindSpeedCoeff(towndistance[i],typhoonintensity[i]))
        return max(towndemand)
    else:
        return 0

def preptime(typhooncoords, typhoonintensity, DPcoord, forecasthour):
    '''Inputs the DP coordinate, and the path of the typhoon (location and intensity). It returns the minimum preparation time required'''
    k = 0
    while k < len(typhooncoords):
        if distance(typhooncoords[k], DPcoord)*1.60934 <= r33(typhoonintensity[k]):
            timeprep = math.floor(forecasthour[k]/6)*6
            break
        else:
            k += 1
    if k == len(typhooncoords):
#         timeprep = (math.floor(k/10)-1)*6
        timeprep = 1000
    return timeprep

def prep(townloc, typhoontrack):
    '''Calculates minimum preparation time for each location (before typhoon hits)'''
    preptimelist = []
    for i in range(len(townloc)):
        preptimelist.append(
            preptime(listcoordinates(typhoontrack['Latitude'],typhoontrack['Longitude']),
                     listintensity(typhoontrack),
                     townloc[i], 
                     listforecasthour(typhoontrack)))
    return pd.DataFrame(preptimelist)


def demand(townloc, townpop, typhoontrack):
    '''Calculates demand for the DP'''
    DPdemand = []
    for i in range(len(townpop)):
        DPdemand.append(
            round((townpop[i]/5)*DPdemandcoeff(listcoordinates(typhoontrack['Latitude'],typhoontrack['Longitude']),listintensity(typhoontrack),townloc[i])))
    return pd.DataFrame(DPdemand)

def interpolateforecastdata(besttrackdf,steps):
    '''interpolate the points for the forecast data. The number of steps indicate the number of time steps you want to split each duration to.'''
    besttrackdf = besttrackdf.reset_index(drop=True)
    newbesttrackdf = besttrackdf
    for i in range(len(besttrackdf)-1):
        intermediate = pd.DataFrame(columns = list(besttrackdf), index=range(steps-1))
        for col in list(besttrackdf):
            if type(besttrackdf.loc[i,col]) == str:
                diff = (float(besttrackdf.loc[i,col]) - float(besttrackdf.loc[i+1,col]))/steps
            else:
                diff = (besttrackdf.loc[i,col] - besttrackdf.loc[i+1,col])/steps
            for j in range(steps-1):
                if type(besttrackdf.loc[i,col]) == str:
                    intermediate.loc[j,col] = float(besttrackdf.loc[i,col])-diff*(j+1)
                else:
                    intermediate.loc[j,col] = besttrackdf.loc[i,col]-diff*(j+1)
        newbesttrackdf = pd.concat([newbesttrackdf.ix[:i+(steps-1)*i], intermediate, newbesttrackdf.ix[i+(steps-1)*i+1:]]).reset_index(drop=True)
    return newbesttrackdf

def listcoordinates(latitude, longitude):
    '''list the latitude and longitude of the towns'''
    coord = []
    latcoord = latitude.tolist()
    loncoord = longitude.tolist()
    for i in range(len(latcoord)):
        coord.append([float(latcoord[i]),float(loncoord[i])])
        i = i+1
    return coord

def listintensity(DF):
    '''Takes forecast intensity from the dataframe, and returns a list'''
    a = list(DF.Intensity.apply(float))
    return a

def listforecasthour(DF):
    '''Takes forecast intensity from the dataframe, and returns a list'''
    a = list(DF.ForecastHour.apply(float))
    return a

def SupplyDestroyed(typhooncoords, typhoonintensity, DPcoord):
    '''Inputs the DP coordinate, and the path of the typhoon (location and intensity). It returns a datarame which indicates whether supply will be destroyed if located at a point'''
    towndistance = []
    townsupply = []
    for i in range(len(typhooncoords)):
        towndistance.append(distance(typhooncoords[i], DPcoord))
    
    if min(towndistance)*1.60934 <= rmw(typhoonintensity[np.argmin(towndistance)]):
        for i in range(len(towndistance)):
            townsupply.append(WindSpeedCoeff(towndistance[i],typhoonintensity[i]))
    else:
        return 1
    if max(townsupply) >= 0.444:
        return 0
    else:
        return 1

def supply(townloc, typhoontrack):
    '''Determines whether a location is a feasible supply point'''
    Supply = []
    for i in range(len(townloc)):
        Supply.append(
            SupplyDestroyed(listcoordinates(typhoontrack['Latitude'],typhoontrack['Longitude']),listintensity(typhoontrack),townloc[i]))
    return pd.DataFrame(Supply)

def roadsremain(roads, DPs):
    '''Removes the projected impassable roads from the raods dataframe'''
    DPsroadsbroken = DPs.loc[DPs['Supply'] == 0]
    DPsroadsbroken
    roadsremaining = roads.copy()
    roadsbrokenlist = list()
    for i in range(len(roads)):
        if roads.loc[i,'Start Code'] in DPsroadsbroken.Code.tolist() and roads.loc[i,'End Code'] in DPsroadsbroken.Code.tolist():
            roadsbrokenlist.append(i)
    desired_indices = [j for j in range(len((roadsremaining.index))) if j in roadsbrokenlist]
    for i in desired_indices:
        roadsremaining.loc[i,'Distance (km)'] = roadsremaining.loc[i,'Distance (km)']*1.50
#     roadsremaining = roadsremaining.iloc[desired_indices]
#     roadsremaining = roadsremaining.reset_index(drop=True)
    return roadsremaining, desired_indices

def djikstratable(Q,weight):
    '''Prepares the djikstra table for the network'''
    djikstratable = pd.DataFrame(columns = Q.nodes(), index = Q.nodes())
    for i in range(len(Q.nodes())):
        for j in range(len(Q.nodes())):
            try:
                djikstratable.iloc[i,j] = nx.shortest_path_length(Q, source=Q.nodes()[i], target=Q.nodes()[j], weight=weight)
            except:
                djikstratable.iloc[i,j] = np.nan
    return djikstratable

def djikstratablebinary(Q, weight, maxtravel):
    '''Prepares the djikstra table for the network'''
    djikstratable = pd.DataFrame(columns = Q.nodes(), index = Q.nodes())
    for i in range(len(Q.nodes())):
        for j in range(len(Q.nodes())):
            try:
                djikstratable.iloc[i,j] = nx.shortest_path_length(Q, source=Q.nodes()[i], target=Q.nodes()[j], weight=weight)
                if nx.shortest_path_length(Q, source=Q.nodes()[i], target=Q.nodes()[j], weight=weight) <= maxtravel:
                    djikstratable.iloc[i,j] = 1
                else:
                    djikstratable.iloc[i,j] = 0
            except:
                djikstratable.iloc[i,j] = 0
    return djikstratable

def roadsdf(file):
    '''Create the dataframe for the road network based on the CSV input file'''
    roads = pd.read_csv(file)
    roads['startlon'] = ''
    roads['startlat'] = ''
    roads['endlon'] = ''
    roads['endlat'] = ''
    for i in range(len(roads)):
        for j in range(len(towns)):
            if roads['Start Code'][i] == towns['Code'][j]:
                roads['startlon'][i] = towns['longitude'][j]
                roads['startlat'][i] = towns['latitude'][j]
            if roads['End Code'][i] == towns['Code'][j]:
                roads['endlon'][i] = towns['longitude'][j]
                roads['endlat'][i] = towns['latitude'][j]
    return roads

def trackswithdemand(DPs, initialtracklist):
    '''Generates a list of dataframes with 1) viable list of typhoon tracks 2) their corresponding demand and potential supply points'''
    viabletracklist = list()
    DPslist = list()
    potentialtracklist = list()
    for i in range(len(initialtracklist)):
        print('now at %d' % i)
        if len(initialtracklist[i]) > 0:
            if max(initialtracklist[i].Intensity)*1.15078 > 74:
                potentialtracklist.append(initialtracklist[i])
    if potentialtracklist:   
        for i in range(len(potentialtracklist)):
            print('now at %d' % i)
            expandedtrackdf = interpolateforecastdata(potentialtracklist[i], 5)
            newDPs = DPs.copy()
            ActualDemand = demand(listcoordinates(newDPs['latitude'],newDPs['longitude']), newDPs.Population.tolist(), expandedtrackdf)
            if ActualDemand.sum().sum() != 0:
                Supply = supply(listcoordinates(newDPs['latitude'],newDPs['longitude']), expandedtrackdf)
                PrepTime = prep(listcoordinates(newDPs['latitude'],newDPs['longitude']), expandedtrackdf)
                newDPs['ActualDemand'] = ActualDemand
                newDPs['Supply'] = Supply
                newDPs['PrepTime'] = PrepTime
                viabletracklist.append(potentialtracklist[i])
                DPslist.append(newDPs)
    return viabletracklist, DPslist

def roaddamagedjikstralist(DPslist, towns, roads, traveltime, costtable):
    #create graph for the response 
    K = nx.Graph()
    #cost of shipping (per FFP item per km)
    truck = .0631
    roro = 0.209

    #add nodes
    for i in range(len(towns)):
        K.add_node(towns['Code'][i],name=towns['Municipality'][i], pos=(towns['longitude'][i],towns['latitude'][i]),province=towns['Province'][i],population=towns['Population'][i],lon=towns['longitude'][i],lat=towns['latitude'][i])

#     #add edges (seas)
#     for i in range(len(seas)):
#         K.add_edge(int(seas['Origin'][i]),int(seas['Destination'][i]),length=seas['Distance.miles'][i]*1.60934,traveltime=seas['Distance.miles'][i]*1.60934/29.632,vessel='ro-ro',cost=roads['Distance (km)'][i]*roro)    
    
    roadremaindjikstralist = list()
    roadremaincost = list()
    for i in range(len(DPslist)):
        print('now at %d' % i)
        if DPslist[i].Supply.sum() == 133:
            roadremaindjikstralist.append(traveltime)
            roadremaincost.append(costtable)
        else:
            roadsremaining, desired_indices = roadsremain(roads, DPslist[i])
            if desired_indices == []:
                roadremaindjikstralist.append(traveltime)
                roadremaincost.append(costtable)
            else:
                J = K.copy()
                #add edges (roads)
                for i in range(len(roadsremaining)):
                    J.add_edge(int(roadsremaining['Start Code'][i]), int(roadsremaining['End Code'][i]), length=roadsremaining['Distance (km)'][i], traveltime=roadsremaining['Distance (km)'][i]/60, vessel='truck', cost=roadsremaining['Distance (km)'][i]*truck)

                #Calculate travel time from each point based on the Djikstra algorithm
                travelallowed = djikstratable(J,'traveltime')
                roadremaindjikstralist.append(travelallowed)
                remaincost = djikstratable(J,'cost')
                roadremaincost.append(remaincost)

    return roadremaindjikstralist, roadremaincost

def forecastdemand(DPs, forecastlist):
    '''Generates a list of forecasted demand, supply damage, and lead times for each forecast'''
    FClist = list()
    for i in range(len(forecastlist)):
        expandedtrackdf = interpolateforecastdata(forecastlist[i], 5)
        newDPs = DPs.copy()
        ActualDemand = demand(listcoordinates(newDPs['latitude'],newDPs['longitude']), newDPs.Population.tolist(), expandedtrackdf)
#         if ActualDemand.sum().sum() != 0:
        Supply = supply(listcoordinates(newDPs['latitude'],newDPs['longitude']), expandedtrackdf)
        PrepTime = prep(listcoordinates(newDPs['latitude'],newDPs['longitude']), expandedtrackdf)
        newDPs['ActualDemand'] = ActualDemand
        newDPs['Supply'] = Supply
        newDPs['PrepTime'] = PrepTime
        FClist.append(newDPs)
    return FClist

def seasdf(filename):
    seas = pd.read_csv(filename)
    regionviseas = pd.DataFrame(columns=list(seas))
    for i in range(len(seas)):
        for j in range(len(towns)):
            if seas.iloc[i,0] == towns['Code'].tolist()[j]:
                for k in range(len(towns)):
                    if seas.iloc[i,1] == towns['Code'].tolist()[k]:
    #             if seas.iloc[i,1] in roads['Start Code'].tolist():
                        regionviseas = regionviseas.append(seas.iloc[i])
    regionviseas = regionviseas.drop_duplicates()
    regionviseas = regionviseas.reset_index(drop=True)
    return regionviseas

def Fijlistgen(DPslist,DPs):
    Fijlist = list()
    for k in range(len(DPslist)):
        print('now at %d' % k)
        Fij = pd.DataFrame(np.zeros((len(DPs), len(DPs))))
        for i in range(len(DPs)):
            for j in range(len(DPs)):
                Fij.iloc[i,j] = min(DPslist[k].loc[i,'PrepTime'],DPslist[k].loc[j,'PrepTime'])
        Fijlist.append(Fij)
    return Fijlist

# Retrieve Raw Input Data

Raw input data include the network data as well as the potential typhoon tracks

## Network Data

In [3]:
#Import towns data (nodes)
towns = pd.read_csv('municipality_profiles.csv')
towns = towns.loc[towns['Province'] == 'ILOILO']
towns = towns.reset_index(drop = True)

#Import roads data (edges)
roads = roadsdf('road_distancesIloilo.csv')
# seas = seasdf('sea_supply_routes.csv')

## Potential Typhoon Tracks Data

In [4]:
#Open Dataframe for the 100 tracks generated for each period of typhoon Haiyan. There are a total of 23 periods
with open('HAIYAN100', 'rb') as fp:
    compilationtrack = pickle.load(fp)
    compilationfulltrack = pickle.load(fp)

In [5]:
compilationtrack = compilationtrack[:21]
compilationfulltrack = compilationfulltrack[:21]

In [6]:
for i in range(len(compilationfulltrack)):
    print('%d' % i)
    for j in range(len(compilationfulltrack[i])):
        print('%d' % j)
        compilationfulltrack[i][j] = compilationfulltrack[i][j].iloc[18:]

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
2
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
3
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


# Generate Variable Model Input Data

For the optimization model, there are input variables that vary depending on the typhoon track. This includes the demand, locations where supplies are not damaged (candidate supply points), and destroyed road networks. The time by which the typhoon will hit a specific point also varies by track. These values are deterministic for every scenario.

## Demand for Each DP, Candidate Supply Points, and Lead Time Available per Location

Based on the list of potential typhoon tracks, it is first important to assess whether the track will generate demand in the area. If the track does not generate any demand, then there is no point in pre-positioning for this specific typhoon track. Thus, the track is not included in the viable track list.

If the track is included in the viable track list, then the demand at each DP, candidate supply points, and lead time available per location is determined.

In [7]:
#Prepare dataframe for DPs (Demand Points) - Includes the identifier code, municipality name, coordinates, population
DPs = towns.sort(columns='Code')
DPs = DPs.reset_index(drop=True)
DPs = DPs.drop(['Province','Region','Income.Class','City.Class','District','Registered.Voters','Land.Area.hectares'],1)
DPs = DPs[['Municipality','Code','latitude','longitude','Population']]

In [8]:
#Narrow down the potential tracklist into a viable track list (corresponds to the tracks that would generate any demand in the network)
start_time = time.time()


compDPslist = list()
compviablelist = list()
for i in range(len(compilationfulltrack)):
    print('now at group %d' % i)
    viabletracklist, DPslist = trackswithdemand(DPs, compilationfulltrack[i])
    compDPslist.append(DPslist)
    compviablelist.append(viabletracklist)
    
print("--- %s seconds ---" % (time.time() - start_time))

now at group 0
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now at 17
now at 18
now at 19
now at 20
now at 21
now at 22
now at 23
now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 31
now at 32
now at 33
now at 34
now at 35
now at 36
now at 37
now at 38
now at 39
now at 40
now at 41
now at 42
now at 43
now at 44
now at 45
now at 46
now at 47
now at 48
now at 49
now at 50
now at 51
now at 52
now at 53
now at 54
now at 55
now at 56
now at 57
now at 58
now at 59
now at 60
now at 61
now at 62
now at 63
now at 64
now at 65
now at 66
now at 67
now at 68
now at 69
now at 70
now at 71
now at 72
now at 73
now at 74
now at 75
now at 76
now at 77
now at 78
now at 79
now at 80
now at 81
now at 82
now at 83
now at 84
now at 85
now at 86
now at 87
now at 88
now at 89
now at 90
now at 91
now at 92
now at 93
now at 94
now at 95
now at 96
now at 97
now at 98
now a

now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 31
now at 32
now at 33
now at 34
now at 35
now at 36
now at 37
now at 38
now at 39
now at 40
now at 41
now at 42
now at 43
now at 44
now at 45
now at 46
now at 47
now at 48
now at 49
now at 50
now at 51
now at 52
now at 53
now at 54
now at 55
now at 56
now at 57
now at 58
now at 59
now at 60
now at 61
now at 62
now at 63
now at 64
now at 65
now at 66
now at 67
now at 68
now at 69
now at 70
now at 71
now at 72
now at 73
now at 74
now at 75
now at 76
now at group 5
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now at 17
now at 18
now at 19
now at 20
now at 21
now at 22
now at 23
now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 31
now at 32
now at 33
now at 34
now at 35
now at 36
now at 37
now at 38
now at 39
now at 40
now at 41
now at 42
now at 43
now at 44
now at 45
now a

now at 55
now at 56
now at 57
now at 58
now at 59
now at 60
now at 61
now at 62
now at 63
now at 64
now at 65
now at 66
now at 67
now at 68
now at 69
now at 70
now at 71
now at 72
now at 73
now at 74
now at 75
now at 76
now at 77
now at 78
now at 79
now at 80
now at 81
now at 82
now at 83
now at 84
now at 85
now at 86
now at 87
now at 88
now at 89
now at 90
now at 91
now at 92
now at 93
now at 94
now at 95
now at 96
now at 97
now at 98
now at 99
now at 100
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now at 17
now at 18
now at 19
now at 20
now at 21
now at 22
now at 23
now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 31
now at 32
now at 33
now at 34
now at 35
now at 36
now at 37
now at 38
now at 39
now at 40
now at 41
now at 42
now at 43
now at 44
now at 45
now at 46
now at 47
now at 48
now at 49
now at 50
now at 51
now at 52
now at 53
now at 54

now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now at 17
now at 18
now at 19
now at 20
now at 21
now at 22
now at 23
now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 31
now at 32
now at 33
now at 34
now at 35
now at 36
now at 37
now at 38
now at 39
now at 40
now at 41
now at 42
now at 43
now at 44
now at 45
now at 46
now at 47
now at 48
now at 49
now at 50
now at 51
now at 52
now at 53
now at 54
now at 55
now at 56
now at 57
now at 58
now at 59
now at 60
now at 61
now at 62
now at 63
now at 64
now at 65
now at 66
now at 67
now at 68
now at 69
now at 70
now at 71
now at 72
now at 73
now at 74
now at 75
now at 76
now at 77
now at 78
now at 79
now at 80
now at 81
now at 82
now at 83
now at 84
now at 85
now at 86
now at 87
now at 88
now at 89
now at 90
now at 91
now at 92
now at 93
now at 94
now at 95
now at 96
now at 97
now at 98
now at 99
now at 100
now at group 18
n

In [9]:
#Create a list of dataframes of Fij (the minimum time needed for pre-positioning at i given that it caters to j)
compFijlist = list()
for i in range(len(compDPslist)):
    print('now at group %d' % i)
    compFijlist.append(Fijlistgen(compDPslist[i],DPs))

now at group 0
now at group 1
now at group 2
now at group 3
now at 0
now at group 4
now at 0
now at 1
now at group 5
now at 0
now at group 6
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at group 7
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at group 8
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now at 17
now at 18
now at 19
now at 20
now at 21
now at 22
now at 23
now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 31
now at 32
now at group 9
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at group 10
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now 

now at 58
now at 59
now at 60
now at 61
now at 62
now at 63
now at 64
now at 65
now at 66
now at 67
now at 68
now at 69
now at 70
now at 71
now at 72
now at 73
now at 74
now at 75
now at 76
now at 77
now at 78
now at 79
now at 80
now at 81
now at 82
now at 83
now at 84
now at 85
now at 86
now at 87
now at 88
now at 89
now at 90
now at 91
now at 92
now at 93
now at 94
now at 95
now at 96
now at 97
now at 98
now at 99
now at 100


## Remaining Potential Connections

In some cases, the potential track can cause disruptions to the transportation network. The function roaddamagedjikstralist returns a table which determines whether a point is reachable to another point based on the roadnetwork damage.

In [12]:
start_time = time.time()

compcijlist = list()
compmijlist = list()
for i in range(len(compDPslist)):
    print(i)
    mijlist, cijlist = roaddamagedjikstralist(compDPslist[i], towns, roads, traveltime, costtable)
    compcijlist.append(cijlist)
    compmijlist.append(mijlist)

print("--- %s seconds ---" % (time.time() - start_time))

0
1
2
3
now at 0
4
now at 0
now at 1
5
now at 0
6
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
7
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
8
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now at 17
now at 18
now at 19
now at 20
now at 21
now at 22
now at 23
now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 31
now at 32
9
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
10
now at 0
now at 1
now at 2
now at 3
now at 4
now at 5
now at 6
now at 7
now at 8
now at 9
now at 10
now at 11
now at 12
now at 13
now at 14
now at 15
now at 16
now at 17
now at 18
now at 19
now at 20
now at 21
now at 22
now at 23
now at 24
now at 25
now at 26
now at 27
now at 28
now at 29
now at 30
now at 

now at 85
now at 86
now at 87
now at 88
now at 89
now at 90
now at 91
now at 92
now at 93
now at 94
now at 95
now at 96
now at 97
now at 98
now at 99
now at 100
--- 382.18661427497864 seconds ---


# Generate Constant Model Input Parameters

## Generate Shortest Travel Time Table, Travel Time Binary Table, and Cost Table

In [ ]:
pd.set_option('display.max_rows', 500)

In [11]:
Q = nx.Graph()
#cost of shipping (per FFP item per km)
truck = .0631
roro = 0.209

#add nodes
for i in range(len(towns)):
    Q.add_node(towns['Code'][i],name=towns['Municipality'][i], pos=(towns['longitude'][i],towns['latitude'][i]),province=towns['Province'][i],population=towns['Population'][i],lon=towns['longitude'][i],lat=towns['latitude'][i])

#add edges (roads)
for i in range(len(roads)):
    Q.add_edge(int(roads['Start Code'][i]),int(roads['End Code'][i]),length=roads['Distance (km)'][i],traveltime=roads['Distance (km)'][i]/60,vessel='truck',cost=roads['Distance (km)'][i]*truck)

# #add edges (seas)
# for i in range(len(seas)):
#     Q.add_edge(int(seas['Origin'][i]),int(seas['Destination'][i]),length=seas['Distance.miles'][i]*1.60934,traveltime=seas['Distance.miles'][i]*1.60934/29.632,vessel='ro-ro',cost=roads['Distance (km)'][i]*roro)

#Calculate travel time from each point based on the Djikstra algorithm
traveltime = djikstratable(Q,'traveltime')

#Calculate delivery cost per unit from each point based on the Djikstra algorithm
costtable = djikstratable(Q,'cost')

In [ ]:
roads

## Generate initial supply table

In [13]:
RDCloc = [20]
RDCquantity = [100000]
capacityLDC = 7500
capacityRDC = 100000

Supply = pd.DataFrame(np.zeros((len(DPs), 1)))
xiiinit = pd.DataFrame(np.zeros((len(DPs), 1)))
capi = pd.DataFrame(np.zeros((len(DPs), 1)))
Pi = pd.DataFrame(np.zeros((len(DPs), 1)))

for i in range(len(DPs)):
    capi.iloc[i,0] = capacityLDC

for i in range(len(RDCloc)):
    Supply.iloc[RDCloc[i],0]= RDCquantity[i]
    xiiinit.iloc[RDCloc[i],0]= 1
    capi.iloc[RDCloc[i],0] = capacityRDC

## List of all parameters

In [14]:
# max number of facilities
# p = 10

#transportation time from supply node h to supply node i
mhi = traveltime

#transport cost per unit of relief good from supply node h to supply node i
chi = costtable

#quantity of goods at supply node i before pre-positioning action
Qi = Supply

#Time required to open LDC i
uio = 8

#Cost to open LDC i
cio = 33000

#Capacity of LDC i
capi = capi

#Binary value equal to 1 if supply node is open prior to prepositioning action
xiiinit = xiiinit

#Total budget available from prepositioning
B = 3000000

#Pi dictates which LDC is permanent and cannot be changed. At first forecast, Pi = 0 as everything can be changed.
Pi = Pi

#Total incurred cost
Tcost = 0

## Save Parameter Data to File

In [15]:
with open('Case2DataParameters', 'wb') as fp:
    pickle.dump(mhi, fp)
    pickle.dump(chi, fp)
    pickle.dump(Qi, fp)
    pickle.dump(uio, fp)
    pickle.dump(cio, fp)
    pickle.dump(capi, fp)
    pickle.dump(xiiinit, fp)
    pickle.dump(B, fp)
    pickle.dump(Pi, fp)
    pickle.dump(Tcost, fp)




In [16]:
with open('Case2compDPs', 'wb') as fp:
    pickle.dump(compDPslist, fp)
    pickle.dump(compviablelist, fp)

with open('Case2Fijlist', 'wb') as fp:
    pickle.dump(compFijlist, fp)

with open('Case2compcij', 'wb') as fp:
    pickle.dump(compcijlist, fp)

with open('Case2compmij', 'wb') as fp:
    pickle.dump(compmijlist, fp)
